# VeVaPy Model Template <a name="top" />

# Table of Contents 
1. [Instructions](#instructions)
    1. [Parameter Optimization Against TSST Data Sets](#TSSTInstructions)
    2. [Parameter Optimization Against Basal Data Sets](#basalInstructions)
    3. [Running Without Parameter Optimization](#noOptInstructions)
2. [Imports](#imports)
3. [Parameters and Initial Conditions](#params)
4. [Put Raw Data into Arrays](#rawdata)
    1. [Plot Basal Data Sets](#plotBasalData)
    2. [Plot Nelson Data Sets](#plotNelsonData)
5. [Model Function--Includes ODE Solver](#modelfunction)
6. [Cost Function Definition](#cost)
7. [Run the Optimization](#run)
8. [Save Output to File](#saveoutput)
9. [Compute Means and Std Devations of Parameters and Output as Table](#paramtable)
10. [Plots](#plots)
11. [No Optimization Simulations](#noOpt)
12. [Run Optimizations for Multiple Patients](#runMultiple)
13. [Dependencies](#dependencies)

# Instructions <a name="instructions"></a>

## Parameter Optimization Against TSST Data Sets <a name="TSSTInstructions" />

**Note:** To quickly run a cell (or a selection of cells), use the shortcut Shift+Enter (or you can also use the button labeled "Run" in the toolbar at the top).

To run simulations with parameter optimization against TSST data, there is no need to change any cells until the heading **Run the Optimization**. Simply run all cells up to the cell below that heading.

In order to set which data set to optimize parameters against, look for the following line of code:
    
    data_to_match = [nelson.ACTH[:,0], nelson.ACTH[:,1], nelson.cortisol[:,0], nelson.cortisol[:,1]]

In order to run against a patient from the TSST data sets, simply change the list entries to reflect the patient number and subject group. The subject groups are:

- nelson.melancholicACTH & nelson.melancholicCortisol (15 patients)
- nelson.atypicalACTH & nelson.atypicalCortisol (14 patients)
- nelson.neitherACTH & nelson.neitherCortisol (14 patients)
- nelson.healthyACTH & nelson.healthyCortisol (15 patients)

You could also run against the mean of all patients cortisol and ACTH concentration values by using `nelson.ACTH[:,1]` and `nelson.cortisol[:,1]`. Or you can run against the mean of any subgroup using `nelson.<subgroup name>Cortisol_mean[:,1]` and `nelson.<subgroup name>ACTH_mean[:,1]` (for instance `nelson.melancholicCortisol_mean[:,1]` & `nelson.melancholicACTH_mean[:,1]`). 

Note that the first column in each data set is the time steps, so indexing with `[:,0]` is referring to the time. These are the values we need to set as the first (ACTH time steps) and third (cortisol time steps) indices of the `data_to_match` list.

The following are several examples of lists you could use for parameter optimization with explanations:

- `nelson.melancholicACTH[:,0], nelson.melancholicACTH[:,1], nelson.melancholicCortisol[:,0], nelson.melancholicCortisol[:,1]`
    - The 1st patient in the Melancholic subgroup
- `nelson.atypicalACTH[:,0], nelson.atypicalACTH[:,14], nelson.atypicalCortisol[:,0], nelson.atypicalCortisol[:,14]`
    - The 14th patient in the Atypical subgroup
- `nelson.healthyACTH[:,0], nelson.healthyACTH[:,2], nelson.healthyCortisol[:,0], nelson.healthyCortisol[:,2]`
    - The 2nd patient in the Healthy Control group
- `nelson.ACTH[:,0], nelson.ACTH[:,1], nelson.cortisol[:,0], nelson.cortisol[:,1]`
    - The mean data set for all patients (depressed and control)
- `nelson.healthyACTH_mean[:,0], nelson.healthyACTH_mean[:,1], nelson.healthyCortisol_mean[:,0], nelson.healthyCortisol_mean[:,1]`
    - The mean of all control patients
    
Next, you need to decide whether you will optimize any initial conditions (ICs). This can be modified in the function `cost_fun(params)`. In the cost function, we use the first two optimized parameters in the list returned by the optimization algorithm to set the ICs we want to optimize (CRH and GR in this example):

    y0 = [params[0], y0[1], y0[2], params[1]]

We then need to pass only the remaining parameters in the list to the model, along with the updated ICs in y0:

    simData = model(params[2:], y0)
    
If you want to not optimize any ICs, you would simply comment out the two lines above in `cost_fun()`, and uncomment the line:

    #simData = model(params, y0)
    
In that case, you will likely want to change the ICs for CRH and GR, as they will stay the same for every iteration. Under the heading **Run the Optimization**, the following line should be modified to reflect the desired ICs:

    y0 = [0, data_to_match[1][0], data_to_match[3][0], 0]
    
Be sure to leave the 2nd and 3rd indices unchanged, as these set the ICs to use the initial values of the real-world data set being matched.

At this point, you are ready to run the optimization, so simply run the cells up to the heading **Save Output to File**. This may take some time, so while it is running you can move on to the next steps (if you run a cell while another is processing, it will add it to a queue).

**Note:** You also have the option of using a cost function based on the maximum distance between simulation and real-world data. Simply change SSE_cost to MAX_cost, the instructions for function arguments remain the same.

The cell directly under the heading **Save Output to File** can be changed so that the root filename matches the simulations being run. This root will be used to save all of the various data and figures generated. The current naming scheme would save the files for 5 iterations of parameter optimization against the mean data set from the Nelson with ICs for CRH and GR optimized as:

    filename_root = "sriramModel_output/nelson-patientMean-5-iterations-ICOpt"

This saves the files in a subfolder specific to this model, which helps keep files organized when running multiple models.

The next few cells create an Excel file containing all of the concentration data and optimized parameter values, and text files containing the initial conditions, parameter bounds and parameter means +- standard deviation across the 5 iterations.

The final step after saving the outputs is to plot the simulations against the real-world data. The cell under the heading **Plots** creates an instance of the Visualizer object from the VeVaPy module called visualize. This will start a dialog which asks for several inputs to generate figures as desired.

After initialization of the object, run its method `make_graphs()`, and it will generate figures using the data you have specified. There are a number of arguments that can be optionally specified for this method, and you can see the recommended values for these in the following function call:

    grapher.make_graphs(std_dev = True, 
                sims_line_labels = ["CRH Simulation", "ACTH Simulation", "Cortisol Simulation", "GR Simulation"], 
                sims_line_colors = ["blue", "blue", "blue", "blue"],
                std_labels = ["Mean +- Standard Deviation", "Mean +- Standard Deviation", "Mean +- Standard Deviation", "Mean +- Standard Deviation"],
                real_data_labels = ["Nelson ACTH - Patient Mean", "Nelson ACTH - Patient Mean", "Nelson Cortisol - Patient Mean"], 
                graph_titles = ["CRH Concentration", "ACTH Concentration", "Cortisol Concentration", "GR Concentration"], 
                xaxis_labels = ["Time (h)", "Time (h)", "Time (h)", "Time (h)"],
                yaxis_labels = ["CRH (micrograms/dL)", "ACTH (pg/mL)", "Cortisol (micrograms/dL)", "GR (micrograms)"],
                legend_locs = ["upper right", "upper right", "upper right", "upper left"],
                figure_size = (25,40),
                savefile=filename_root+".png")

## Parameter Optimization Against Basal Data Sets <a name="basalInstructions" />

Since these data sets have data points over a 24-hour period, rather than 2.35 hours, you will need to change the time interval over which the ODE solver integrates. To do this, go to the cell directly above the heading **Put Raw Data Into Arrays** and change the value of t_end to the following:

    t_end = 24.26

The reason you add the extra 0.26 hours is that you need to make sure that when you interpolate between your simulated data points the line covers every real-world data point so that you don't cause issues when computing the cost function (and the last data point for the Golier cortisol concentration data sets is at 24.25 hours).

After making this change, you need to again change the `data_to_match` list so that you are matching the basal data set in which you are interested.

First, choose which data set you wish to match. Here are the options:

- yehuda.controlCortisol
- yehuda.PTSDCortisol
- yehuda.depressedCortisol
- carroll.controlCortisol & carroll.controlACTH
- carroll.LCDepressedCortisol & carroll.LCDepressedACTH (LC = Low Cortisol)
- carroll.HCDepressedCortisol & carroll.HCDepressedACTH (HC = High Cortisol)
- golier.PTSDCortisol & golierPTSDACTH
- golier.nonPTSDTraumaExposedCortisol & golier.nonPTSDTraumaExposedACTH
- golier.nonPTSDNonExposedCortisol & golier.nonPTSDNonExposedACTH
- bremner.abusedPTSDCortisol
- bremner.nonAbusedPTSDCortisol
- bremner.nonAbusedNonPTSDCortisol

**Note:** To see what any of these data sets looks like, click on the **Plot Basal Data Sets** heading in the Table of Contents.

**Note Also:** These data sets all come in smoothed versions (each data point is set to the average of the nearest 5 points of the unsmoothed data). Also, the data sets by Carroll, Golier and Bremner also come in rearranged (or smoothed & rearranged) versions to match the starting time of the Yehuda data (10AM). To use any of these versions, simply append one of the following tags to the end of the data set name (before the indices): `_smooth`, `_rearr`, or `_rearr_smooth`.

First, I will cover what to do with data sets that contain both ACTH and cortisol values, and then afterwards I will cover using the Yehuda and Bremner data sets (which have only cortisol concentration data). Just as with the Nelson data, in all of these data sets the first column is the time step values. This means that if you take any of these arrays and index it with `[:,0]`, you are referring to the time steps. These are the values we need to use as the first (ACTH time steps) and third (cortisol time steps) indices in the `data_to_match` list.

Then for the second and fourth indices, you index the same data sets with `[:,1]` to mean the second column (which contains the mean concentration values for each patient group).

Here are a couple of examples showing lists you can use for optimization:

- `carroll.controlACTH_smooth[:,0], carroll.controlACTH_smooth[:,1], carroll.controlCortisol_smooth[:,0], carroll.controlCortisol_smooth[:,1]`
    - The smoothed Control group mean for the Carroll data set
- `golier.nonPTSDTraumaExposedACTH[:,0], golier.nonPTSDTraumaExposedACTH[:,1], golier.nonPTSDTraumaExposedCortisol[:,0], golier.nonPTSDTraumaExposedCortisol[:,1]`
    - The Trauma-Exposed Control group mean for the Golier data set
    
In order to run simulations against data sets that do not include ACTH concentration data, you will need to change the name of the cost function to `optimize.SSE_cost_noACTH` and then update `data_to_match` to not include the two arguments for ACTH data. To use the Yehuda Control group data, this would look like:

    data_to_match = [yehuda.controlCortisol[:,0], yehuda.controlCortisol[:,1]]
    return optimize.SSE_cost_noACTH(data_to_match[0], data_to_match[1], simData)

For data without ACTH concentrations, you will also need to comment out the current definition of `y0` and uncomment the following line (and change the ICs to the desired values for CRH and ACTH):

    #y0 = [0, 10, data_to_match[1][0], 0]
    
At this point, you're ready to run the parameter optimization, so run all of the cells under the heading **Run the Optimization**.

The cell directly under the heading **Save Output to File** should again have the filename changed to something that reflects the data set you're matching now. For instance, the filename root when matching the smoothed Carroll Control group and optimizing ICs would become:

    filename_root = 'sriramModel_output/carroll-control-smooth-5-iterations-ICopt'
            
Finally, the cells under the heading **Plots** should be run again to generate graphs. The same process of giving inputs to the object dialog will be performed and then the method `make_graphs()` should be run with any optional arguments desired.

## Running Without Parameter Optimization <a name="noOptInstructions" />

To run the model with any set of paramaters you desire, without optimization, you can use the cells under the heading **No Optimization Run**. Change the parameters and initial conditions defined under the heading **Parameters and Initial Conditions**, and then run the cell containing the following line:

    data_no_opt = model(authors_params, y0)
    
Then run the cells under **Plots** to create graphs as described in the sections regarding simulations with parameter optimization.

[Back to Top](#top)

# Imports <a name="imports"></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as sco
from scipy.interpolate import interp1d
import mpld3
from tabulate import tabulate
import matplotlib
import pandas as pd
from VeVaPy import DEsolver, optimize
from VeVaPy.dataImport import data
from VeVaPy.visualize import Visualizer
import seaborn as sns

[Back to Top](#top)

# Parameters and Initial Conditions <a name="params"/>

In [ ]:
# Set the Initial Conditions 
#  (The current array is for a standard model with equations for CRH, ACTH and cortisol, in that order)
# We will use the starting concentrations in ACTH and cortisol from the data set we are matching

y0 = [1, 20, 6]

In [ ]:
# Set the authors' parameter values for running simulations without optimization (for verification purposes)

parameter1 = 1
parameter2 = 2

In [ ]:
# Create an array with the parameter values included, so we can pass them all to the model in a single function argument

authors_params = [parameter1, parameter2]

In [ ]:
# Define bounds on parameters (for optimization during validation procedure)
# The example array below means that parameter1 has a lower bound of 0 and an upper bound of 1,
# and parameter2 has a lower bound of 1 and an upper bound of 2
bounds = [(0, 1), (1, 2)]

In [ ]:
# Time interval and step definition

# For models with time scale in hours, matching TSST data
t_start = -0.01
t_end = 2.35
t_step = 0.01

# For models with time scale in hours, matching basal 24-hour data
#t_start = -0.01
#t_end = 24.01
#t_step = 0.01

# For models with time scale in minutes, matching TSST data
#t_start = -0.01
#t_end = 140.01
#t_step = 0.01

# For models with time scale in minutes, matching basal 24-hour data
#t_start = -0.01
#t_end = 1440.01
#t_step = 0.01

# NOTE: We start at -0.01 to ensure that we have a time step covering exactly 0.0, for comparison with the 
#  real-world data. Otherwise, starting at 0.0 our first simulated point would be at 0.001 or so.

[Back to Top](#top)

# Put Raw Data Into Arrays <a name="rawdata"/>

In [ ]:
# Create an instance of the data class for each data set contained in the HPAmodeling library, and set the time
#  scale to match the model you're using.
yehuda = data("yehuda", "hours")
carroll = data("carroll", "hours")
golier = data("golier", "hours")
bremner = data("bremner", "hours")
nelson = data("nelson", "hours")

[Back to Top](#top)

## Plot Basal Data Sets <a name="plotBasalData"></a>

In [ ]:
%matplotlib inline
font = {'size'   : 18}
matplotlib.rc('font', **font)

fig, (ax1, ax2, ax3) = plt.subplots(nrows = 3, figsize = (20,20))

ax1.plot(yehuda.controlCortisol[:,0], yehuda.controlCortisol[:,1], label = "Control Group Cortisol")
ax1.plot(yehuda.controlCortisol_smooth[:,0], yehuda.controlCortisol_smooth[:,1], label = "Control Group Cortisol - Smoothed")
ax1.set(xlabel="Time (hours)", ylabel="Cortisol (micrograms/dL)")
ax1.legend(loc="lower right", shadow = True, fancybox = True)

ax2.plot(yehuda.PTSDCortisol[:,0], yehuda.PTSDCortisol[:,1], label = "PTSD Group Cortisol")
ax2.plot(yehuda.PTSDCortisol_smooth[:,0], yehuda.PTSDCortisol_smooth[:,1], label = "PTSD Group Cortisol - Smoothed")
ax2.set(xlabel="Time (hours)", ylabel="Cortisol (micrograms/dL)")
ax2.legend(loc="lower right", shadow = True, fancybox = True)

ax3.plot(yehuda.depressedCortisol[:,0], yehuda.depressedCortisol[:,1], label = "Depression Group Cortisol")
ax3.plot(yehuda.depressedCortisol_smooth[:,0], yehuda.depressedCortisol_smooth[:,1], label = "Depression Group Cortisol - Smoothed")
ax3.set(xlabel="Time (hours)", ylabel="Cortisol (micrograms/dL)")
ax3.legend(loc="lower right", shadow = True, fancybox = True)


In [ ]:
#mpld3.enable_notebook()
%matplotlib inline

font = {'size'   : 20}
matplotlib.rc('font', **font)
fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows = 4, figsize = (25,25))

ax1.plot(carroll.controlCortisol_rearr[:,0], carroll.controlCortisol_rearr[:,1], 'b', label = "Control")
ax1.plot(carroll.HCDepressedCortisol_rearr[:,0], carroll.HCDepressedCortisol_rearr[:,1], 'r', label = "High Cortisol Depressed")
ax1.plot(carroll.controlCortisol_rearr_smooth[:,0], carroll.controlCortisol_rearr_smooth[:,1], label = "Control - Smoothed")
ax1.plot(carroll.HCDepressedCortisol_rearr_smooth[:,0], carroll.HCDepressedCortisol_rearr_smooth[:,1], label = "High Cortisol Depressed - Smoothed")
ax1.set(xlabel="Time (hours)", ylabel="Cortisol (micrograms/dL)")
ax1.legend(loc="upper right", shadow = True, fancybox = True)

ax2.plot(carroll.controlCortisol_rearr[:,0], carroll.controlCortisol_rearr[:,1], 'b', label = "Control")
ax2.plot(carroll.LCDepressedCortisol_rearr[:,0], carroll.LCDepressedCortisol_rearr[:,1], 'g', label = "Low Cortisol Depressed")
ax2.plot(carroll.controlCortisol_rearr_smooth[:,0], carroll.controlCortisol_rearr_smooth[:,1], label = "Control - Smoothed")
ax2.plot(carroll.LCDepressedCortisol_rearr_smooth[:,0], carroll.LCDepressedCortisol_rearr_smooth[:,1], label = "Low Cortisol Depressed - Smoothed")
ax2.set(xlabel="Time (hours)", ylabel="Cortisol (micrograms/dL)")
ax2.legend(loc="upper right", shadow = True, fancybox = True)

ax3.plot(carroll.controlACTH_rearr[:,0], carroll.controlACTH_rearr[:,1], 'b', label = "Control")
ax3.plot(carroll.HCDepressedACTH_rearr[:,0], carroll.HCDepressedACTH_rearr[:,1], 'r', label = "High Cortisol Depressed")
ax3.plot(carroll.controlACTH_rearr_smooth[:,0], carroll.controlACTH_rearr_smooth[:,1], label = "Control - Smoothed")
ax3.plot(carroll.HCDepressedACTH_rearr_smooth[:,0], carroll.HCDepressedACTH_rearr_smooth[:,1], label = "High Cortisol Depressed - Smoothed")
ax3.set(xlabel="Time (hours)", ylabel="ACTH (pg/mL)")
ax3.legend(loc="upper right", shadow = True, fancybox = True)

ax4.plot(carroll.controlACTH_rearr[:,0], carroll.controlACTH_rearr[:,1], 'b', label = "Control")
ax4.plot(carroll.LCDepressedACTH_rearr[:,0], carroll.LCDepressedACTH_rearr[:,1], 'g', label = "Low Cortisol Depressed")
ax4.plot(carroll.controlACTH_rearr_smooth[:,0], carroll.controlACTH_rearr_smooth[:,1], label = "Control - Smoothed")
ax4.plot(carroll.LCDepressedACTH_rearr_smooth[:,0], carroll.LCDepressedACTH_rearr_smooth[:,1], label = "Low Cortisol Depressed - Smoothed")
ax4.set(xlabel="Time (hours)", ylabel="ACTH (pg/mL)")
ax4.legend(loc="upper right", shadow = True, fancybox = True)

In [ ]:
%matplotlib inline

fig, (ax1, ax2, ax3, ax4, ax5, ax6) = plt.subplots(nrows = 6, figsize = (15,20))

ax1.plot(golier.PTSDCortisol_rearr_smooth[:,0], golier.PTSDCortisol_rearr_smooth[:,1], label = "Trauma Exposed PTSD Cortisol - Smoothed")
ax1.plot(golier.PTSDCortisol_rearr[:,0], golier.PTSDCortisol_rearr[:,1], label = "Trauma Exposed PTSD Cortisol")
ax1.set(xlabel="Time (hours)", ylabel="Cortisol (mg/dL)")
ax1.legend(loc="lower right", shadow = True, fancybox = True)

ax2.plot(golier.nonPTSDTraumaExposedCortisol_rearr_smooth[:,0], golier.nonPTSDTraumaExposedCortisol_rearr_smooth[:,1], label = "Trauma Exposed Non-PTSD Cortisol - Smoothed")
ax2.plot(golier.nonPTSDTraumaExposedCortisol_rearr[:,0], golier.nonPTSDTraumaExposedCortisol_rearr[:,1], label = "Trauma Exposed Non-PTSD Cortisol")
ax2.set(xlabel="Time (hours)", ylabel="Cortisol (mg/dL)")
ax2.legend(loc="lower right", shadow = True, fancybox = True)

ax3.plot(golier.nonPTSDNonExposedCortisol_rearr_smooth[:,0], golier.nonPTSDNonExposedCortisol_rearr_smooth[:,1], label = "Non-Exposed Non-PTSD Cortisol - Smoothed")
ax3.plot(golier.nonPTSDNonExposedCortisol_rearr[:,0], golier.nonPTSDNonExposedCortisol_rearr[:,1], label = "Non-Exposed Non-PTSD Cortisol")
ax3.set(xlabel="Time (hours)", ylabel="Cortisol (mg/dL)")
ax3.legend(loc="lower right", shadow = True, fancybox = True)

ax4.plot(golier.PTSDACTH_rearr_smooth[:,0], golier.PTSDACTH_rearr_smooth[:,1], label = "Trauma Exposed PTSD ACTH - Smoothed")
ax4.plot(golier.PTSDACTH_rearr[:,0], golier.PTSDACTH_rearr[:,1], label = "Trauma Exposed PTSD ACTH")
ax4.set(xlabel="Time (hours)", ylabel="ACTH (pg/mL)")
ax4.legend(loc="lower right", shadow = True, fancybox = True)

ax5.plot(golier.nonPTSDTraumaExposedACTH_rearr_smooth[:,0], golier.nonPTSDTraumaExposedACTH_rearr_smooth[:,1], label = "Trauma Exposed Non-PTSD ACTH - Smoothed")
ax5.plot(golier.nonPTSDTraumaExposedACTH_rearr[:,0], golier.nonPTSDTraumaExposedACTH_rearr[:,1], label = "Trauma Exposed Non-PTSD ACTH")
ax5.set(xlabel="Time (hours)", ylabel="ACTH (pg/mL)")
ax5.legend(loc="lower right", shadow = True, fancybox = True)

ax6.plot(golier.nonPTSDNonExposedACTH_rearr_smooth[:,0], golier.nonPTSDNonExposedACTH_rearr_smooth[:,1], label = "Non-Exposed Non-PTSD ACTH - Smoothed")
ax6.plot(golier.nonPTSDNonExposedACTH_rearr[:,0], golier.nonPTSDNonExposedACTH_rearr[:,1], label = "Non-Exposed Non-PTSD ACTH")
ax6.set(xlabel="Time (hours)", ylabel="ACTH (pg/mL)")
ax6.legend(loc="lower right", shadow = True, fancybox = True)


In [ ]:
%matplotlib inline

fig, (ax1, ax2, ax3) = plt.subplots(nrows = 3, figsize = (15,15))

ax1.plot(bremner.abusedPTSDCortisol_rearr_smooth[:,0], bremner.abusedPTSDCortisol_rearr_smooth[:,1], label = "Abused PTSD Cortisol - Smoothed")
ax1.plot(bremner.abusedPTSDCortisol_rearr[:,0], bremner.abusedPTSDCortisol_rearr[:,1], label = "Abused PTSD Cortisol")
ax1.set(xlabel="Time (hours)", ylabel="Cortisol (microg/dL)")
ax1.legend(loc="lower right", shadow = True, fancybox = True)

ax2.plot(bremner.nonAbusedPTSDCortisol_rearr_smooth[:,0], bremner.nonAbusedPTSDCortisol_rearr_smooth[:,1], label = "Non-Abused PTSD Cortisol - Smoothed")
ax2.plot(bremner.nonAbusedPTSDCortisol_rearr[:,0], bremner.nonAbusedPTSDCortisol_rearr[:,1], label = "Non-Abused PTSD Cortisol")
ax2.set(xlabel="Time (hours)", ylabel="Cortisol (microg/dL)")
ax2.legend(loc="lower right", shadow = True, fancybox = True)

ax3.plot(bremner.nonAbusedNonPTSDCortisol_rearr_smooth[:,0], bremner.nonAbusedNonPTSDCortisol_rearr_smooth[:,1], label = "Non-Abused Non-PTSD Cortisol - Smoothed")
ax3.plot(bremner.nonAbusedNonPTSDCortisol_rearr[:,0], bremner.nonAbusedNonPTSDCortisol_rearr[:,1], label = "Non-Abused Non-PTSD Cortisol")
ax3.set(xlabel="Time (hours)", ylabel="Cortisol (microg/dL)")
ax3.legend(loc="lower left", shadow = True, fancybox = True)


[Back to Top](#top)

## Plot Nelson Data Sets <a name="plotNelsonData"></a>

In [ ]:
matplotlib.rc('font', **{'size'   : 28})

fig, (ax1, ax2) = plt.subplots(nrows = 2, figsize = (15,20))

ax1.plot(nelson.ACTH[:,0], nelson.ACTH[:,3], label = "Mean ACTH")
ax1.legend(loc = "upper right", shadow = True, fancybox = True)
ax1.set(xlabel = "Time (h)", ylabel = "ACTH (pg/mL)", title = "ACTH Concentration")

ax2.plot(nelson.cortisol[:,0], nelson.cortisol[:,3], label = "Mean Cortisol")
ax2.legend(loc = "upper right", shadow = True, fancybox = True)
ax2.set(xlabel = "Time (h)", ylabel = "Cortisol (micrograms/dL)", title = "Cortisol Concentration")

In [ ]:
# graph the Nelson data (only showing one patient at a time currently)
matplotlib.rc('font', **{'size'   : 12})

fig, (ax1, ax2) = plt.subplots(nrows = 2, figsize = (15,20))

#ax1.plot(nelsonACTH[:,0], nelsonACTH[:,1])
ax1.plot(nelson.atypicalACTH[:,0], nelson.atypicalACTH[:,1], label = "Patient 1")
ax1.plot(nelson.atypicalACTH[:,0], nelson.atypicalACTH[:,2], label = "Patient 2")
ax1.plot(nelson.atypicalACTH[:,0], nelson.atypicalACTH[:,3], label = "Patient 3")
ax1.plot(nelson.atypicalACTH[:,0], nelson.atypicalACTH[:,4], label = "Patient 4")
ax1.plot(nelson.atypicalACTH[:,0], nelson.atypicalACTH[:,5], label = "Patient 5")
ax1.plot(nelson.atypicalACTH[:,0], nelson.atypicalACTH[:,6], label = "Patient 6")
ax1.plot(nelson.atypicalACTH[:,0], nelson.atypicalACTH[:,7], label = "Patient 7")
ax1.plot(nelson.atypicalACTH[:,0], nelson.atypicalACTH[:,8], label = "Patient 8")
ax1.plot(nelson.atypicalACTH[:,0], nelson.atypicalACTH[:,9], label = "Patient 9")
ax1.plot(nelson.atypicalACTH[:,0], nelson.atypicalACTH[:,10], label = "Patient 10")
ax1.plot(nelson.atypicalACTH[:,0], nelson.atypicalACTH[:,11], label = "Patient 11")
ax1.plot(nelson.atypicalACTH[:,0], nelson.atypicalACTH[:,12], label = "Patient 12")
ax1.plot(nelson.atypicalACTH[:,0], nelson.atypicalACTH[:,13], label = "Patient 13")
ax1.plot(nelson.atypicalACTH[:,0], nelson.atypicalACTH[:,14], label = "Patient 14")
ax1.legend(loc = "upper right", shadow = True, fancybox = True)
ax1.set(xlabel = "Time (h)", ylabel = "ACTH (pg/mL)", title = "ACTH Concentration")

#ax2.plot(nelsonCortisol[:,0], nelsonCortisol[:,1])
ax2.plot(nelson.atypicalCortisol[:,0], nelson.atypicalCortisol[:,1], label = "Patient 1")
ax2.plot(nelson.atypicalCortisol[:,0], nelson.atypicalCortisol[:,2], label = "Patient 2")
ax2.plot(nelson.atypicalCortisol[:,0], nelson.atypicalCortisol[:,3], label = "Patient 3")
ax2.plot(nelson.atypicalCortisol[:,0], nelson.atypicalCortisol[:,4], label = "Patient 4")
ax2.plot(nelson.atypicalCortisol[:,0], nelson.atypicalCortisol[:,5], label = "Patient 5")
ax2.plot(nelson.atypicalCortisol[:,0], nelson.atypicalCortisol[:,6], label = "Patient 6")
ax2.plot(nelson.atypicalCortisol[:,0], nelson.atypicalCortisol[:,7], label = "Patient 7")
ax2.plot(nelson.atypicalCortisol[:,0], nelson.atypicalCortisol[:,8], label = "Patient 8")
ax2.plot(nelson.atypicalCortisol[:,0], nelson.atypicalCortisol[:,9], label = "Patient 9")
ax2.plot(nelson.atypicalCortisol[:,0], nelson.atypicalCortisol[:,10], label = "Patient 10")
ax2.plot(nelson.atypicalCortisol[:,0], nelson.atypicalCortisol[:,11], label = "Patient 11")
ax2.plot(nelson.atypicalCortisol[:,0], nelson.atypicalCortisol[:,12], label = "Patient 12")
ax2.plot(nelson.atypicalCortisol[:,0], nelson.atypicalCortisol[:,13], label = "Patient 13")
ax2.plot(nelson.atypicalCortisol[:,0], nelson.atypicalCortisol[:,14], label = "Patient 14")
ax2.legend(loc = "upper right", shadow = True, fancybox = True)
ax2.set(xlabel = "Time (h)", ylabel = "Cortisol (micrograms/dL)", title = "Cortisol Concentration")

In [ ]:
# graph the Nelson data (only showing one patient at a time currently)
matplotlib.rc('font', **{'size'   : 12})

fig, (ax1, ax2) = plt.subplots(nrows = 2, figsize = (15,20))

#ax1.plot(nelsonACTH[:,0], nelsonACTH[:,1])
ax1.plot(nelson.melancholicACTH[:,0], nelson.melancholicACTH[:,1], label = "Patient 1")
ax1.plot(nelson.melancholicACTH[:,0], nelson.melancholicACTH[:,2], label = "Patient 2")
ax1.plot(nelson.melancholicACTH[:,0], nelson.melancholicACTH[:,3], label = "Patient 3")
ax1.plot(nelson.melancholicACTH[:,0], nelson.melancholicACTH[:,4], label = "Patient 4")
ax1.plot(nelson.melancholicACTH[:,0], nelson.melancholicACTH[:,5], label = "Patient 5")
ax1.plot(nelson.melancholicACTH[:,0], nelson.melancholicACTH[:,6], label = "Patient 6")
ax1.plot(nelson.melancholicACTH[:,0], nelson.melancholicACTH[:,7], label = "Patient 7")
ax1.plot(nelson.melancholicACTH[:,0], nelson.melancholicACTH[:,8], label = "Patient 8")
ax1.plot(nelson.melancholicACTH[:,0], nelson.melancholicACTH[:,9], label = "Patient 9")
ax1.plot(nelson.melancholicACTH[:,0], nelson.melancholicACTH[:,10], label = "Patient 10")
ax1.plot(nelson.melancholicACTH[:,0], nelson.melancholicACTH[:,11], label = "Patient 11")
ax1.plot(nelson.melancholicACTH[:,0], nelson.melancholicACTH[:,12], label = "Patient 12")
ax1.plot(nelson.melancholicACTH[:,0], nelson.melancholicACTH[:,13], label = "Patient 13")
ax1.plot(nelson.melancholicACTH[:,0], nelson.melancholicACTH[:,14], label = "Patient 14")
ax1.legend(loc = "upper left", shadow = True, fancybox = True)
ax1.set(xlabel = "Time (h)", ylabel = "ACTH (pg/mL)", title = "ACTH Concentration")

#ax2.plot(nelsonCortisol[:,0], nelsonCortisol[:,1])
ax2.plot(nelson.melancholicCortisol[:,0], nelson.melancholicCortisol[:,1], label = "Patient 1")
ax2.plot(nelson.melancholicCortisol[:,0], nelson.melancholicCortisol[:,2], label = "Patient 2")
ax2.plot(nelson.melancholicCortisol[:,0], nelson.melancholicCortisol[:,3], label = "Patient 3")
ax2.plot(nelson.melancholicCortisol[:,0], nelson.melancholicCortisol[:,4], label = "Patient 4")
ax2.plot(nelson.melancholicCortisol[:,0], nelson.melancholicCortisol[:,5], label = "Patient 5")
ax2.plot(nelson.melancholicCortisol[:,0], nelson.melancholicCortisol[:,6], label = "Patient 6")
ax2.plot(nelson.melancholicCortisol[:,0], nelson.melancholicCortisol[:,7], label = "Patient 7")
ax2.plot(nelson.melancholicCortisol[:,0], nelson.melancholicCortisol[:,8], label = "Patient 8")
ax2.plot(nelson.melancholicCortisol[:,0], nelson.melancholicCortisol[:,9], label = "Patient 9")
ax2.plot(nelson.melancholicCortisol[:,0], nelson.melancholicCortisol[:,10], label = "Patient 10")
ax2.plot(nelson.melancholicCortisol[:,0], nelson.melancholicCortisol[:,11], label = "Patient 11")
ax2.plot(nelson.melancholicCortisol[:,0], nelson.melancholicCortisol[:,12], label = "Patient 12")
ax2.plot(nelson.melancholicCortisol[:,0], nelson.melancholicCortisol[:,13], label = "Patient 13")
ax2.plot(nelson.melancholicCortisol[:,0], nelson.melancholicCortisol[:,14], label = "Patient 14")
ax2.legend(loc = "upper left", shadow = True, fancybox = True)
ax2.set(xlabel = "Time (h)", ylabel = "Cortisol (micrograms/dL)", title = "Cortisol Concentration")

In [ ]:
%matplotlib inline

matplotlib.rc('font', **{'size'   : 28})

fig, (ax1, ax2) = plt.subplots(nrows = 2, figsize = (15,20))

ax1.plot(nelson.healthyACTH_mean[:,0], nelson.healthyACTH_mean[:,1], label = "Healthy Control Group", color = "blue")
ax1.plot(nelson.depressedACTH_mean[:,0], nelson.depressedACTH_mean[:,1], label = "Depression Group", color = "orange")
ax1.legend(loc = "upper right", shadow = True, fancybox = True)
ax1.set(xlabel = "Time (h)", ylabel = "ACTH (pg/mL)", title = "ACTH Concentration")

ax2.plot(nelson.healthyCortisol_mean[:,0], nelson.healthyCortisol_mean[:,1], label = "Healthy Control Group", color = "blue")
ax2.plot(nelson.depressedCortisol_mean[:,0], nelson.depressedCortisol_mean[:,1], label = "Depression Group", color = "orange")
ax2.legend(loc = "upper right", shadow = True, fancybox = True)
ax2.set(xlabel = "Time (h)", ylabel = "Cortisol (micrograms/dL)", title = "Cortisol Concentration")

#plt.savefig("Nelson Data Visualization Figures/nelson-control-vs-depressed-mean-comparison.png", dpi = 300)

In [ ]:
%matplotlib inline

matplotlib.rc('font', **{'size'   : 28})

fig, (ax1, ax2) = plt.subplots(nrows = 2, figsize = (25,20))

ax1.plot(nelson.melancholicACTH_mean[:,0], nelson.melancholicACTH_mean[:,1], label = "Melancholic Depression Group", color = "blue")
ax1.plot(nelson.atypicalACTH_mean[:,0], nelson.atypicalACTH_mean[:,1], label = "Atypical Depression Group", color = "orange")
ax1.plot(nelson.neitherACTH_mean[:,0], nelson.neitherACTH_mean[:,1], label = "Non-categorized Depression Group", color = "red")
ax1.legend(loc = "upper right", shadow = True, fancybox = True)
ax1.set(xlabel = "Time (h)", ylabel = "ACTH (pg/mL)", title = "ACTH Concentration")

ax2.plot(nelson.melancholicCortisol[:,0], nelson.melancholicCortisol_mean[:,1], label = "Melancholic Depression Group", color = "blue")
ax2.plot(nelson.atypicalCortisol[:,0], nelson.atypicalCortisol_mean[:,1], label = "Atypical Depression Group", color = "orange")
ax2.plot(nelson.neitherCortisol[:,0], nelson.neitherCortisol_mean[:,1], label = "Non-categorized Depression Group", color = "red")
ax2.legend(loc = "upper right", shadow = True, fancybox = True)
ax2.set(xlabel = "Time (h)", ylabel = "Cortisol (micrograms/dL)", title = "Cortisol Concentration")

#plt.savefig("Nelson Data Visualization Figures/nelson-depression-subtypes-mean-comparison.png", dpi = 300)

In [ ]:
%matplotlib inline

matplotlib.rc('font', **{'size'   : 28})

fig, (ax1, ax2) = plt.subplots(nrows = 2, figsize = (20,20))

ax1.plot(nelson.healthyACTH[:,0], nelson.healthyACTH_mean[:,1], label = "Healthy Control Group", color = "blue")
ax1.plot(nelson.melancholicACTH[:,0], nelson.melancholicACTH_mean[:,1], label = "Melancholic Depression Group", color = "orange")
ax1.legend(loc = "upper right", shadow = True, fancybox = True)
ax1.set(xlabel = "Time (h)", ylabel = "ACTH (pg/mL)", title = "ACTH Concentration")

ax2.plot(nelson.healthyCortisol[:,0], nelson.healthyCortisol_mean[:,1], label = "Healthy Control Group", color = "blue")
ax2.plot(nelson.melancholicCortisol[:,0], nelson.melancholicCortisol_mean[:,1], label = "Melancholic Depression Group", color = "orange")
ax2.legend(loc = "upper right", shadow = True, fancybox = True)
ax2.set(xlabel = "Time (h)", ylabel = "Cortisol (micrograms/dL)", title = "Cortisol Concentration")

#plt.savefig("Nelson Data Visualization Figures/nelson-control-vs-melancholic-depression-mean-comparison.png", dpi = 300)

[Back to Top](#top)

# Model Function--Includes ODE Solver <a name="modelfunction"></a>

In [ ]:
def model(params, ics):
    def ode_system(t, y):
        # Set the size of the array to equal the number of equations in the model (currently size 3 for CRH, ACTH, cortisol)
        dy = np.zeros(3)
        
        # Set the parameters from the input to the function (so that the parameter optimization algorithm can pass
        #  parameter values repeatedly). Change this array to include all parameters you want to optimize.
        [parameter1, parameter2] = params
        
        # Change these to reflect the equations of your model. Note that y[0], y[1] and y[2] are the variable 
        #  concentrations in the system and the dy values are the rates of change.
        dy[0] = parameter1*y[0]
        dy[1] = parameter2*y[1]
        dy[2] = y[2]
        
        return dy
    
    # If you are trying to solve a DDE, add the option: delay = [False, True, True]
    # Set the variables that have a delay included to True and leave the others false, so the example above is
    #  for delayed ACTH and cortisol but not CRH.
    # You'll also need to define the delay lengths, so add tau0 = X, tau1 = X and/or tau2 = X depending on which you
    #  are delaying.
    # Also, if you're having trouble running a DDE, and it's going too slowly, try adding the flag delay_rough = True
    timeSeries = DEsolver.solve(ode_system, ics, t_start, t_step, t_end, ode_steps=1000, ode_atol = 1e-8, ode_rtol = 1e-8)
    return timeSeries

# Cost Function Definition <a name="cost"></a>

In [ ]:
def cost_fun(params):
    global y0, data_to_match
    # If you are optimizing any initial conditions, you can define ICs here as follows:
    #y0 = [params[0], y0[1], y0[2]]
    # This example is for a 3-equation system with CRH, ACTH, and cortisol. The first entry of the array bounds
    #  would be bounds on the initial condition for CRH, and so you'd use params[0] in your y0 array
    
    # If you're optimizing any initial conditions, make sure to not pass those entries of the parameter array to model()
    # Instead, you'd pass params[1:] (all entries except the first 1) in the example above
    simData = model(params, y0)
    
    # Change the first 4 arguments here to be: ACTH time steps, ACTH data points, cortisol time steps, and cortisol 
    #  data points for whichever data set you are matching. Note that all data sets included in dataImport have the first
    #  column as the time steps (so [:,0] is the indexing you'd use)
    # If you're matching data without ACTH concentrations, use costFun.SSE_cost_noACTH or costFun.MAX_cost_noACTH and 
    #  use only 2 arguments, one for cortisol time steps and one for cortisol concentrations.
    return optimize.SSE_cost(data_to_match[0], data_to_match[1], data_to_match[2], data_to_match[3], simData)

[Back to Top](#top)

# Run the Optimization <a name="run"></a>

In [ ]:
# Define the data set to match with the parameter optimization algorithm.
# Requires 4 indices, in the order:
# ACTH time steps, ACTH data, Cortisol time steps, Cortisol data
data_to_match = [nelson.ACTH[:,0], nelson.ACTH[:,1], nelson.cortisol[:,0], nelson.cortisol[:,1]]

# For matching data with only cortisol concentrations, use the following line and change the data sets as desired:
#data_to_match = [yehuda.controlCortisol[:,0], yehuda.controlCortisol[:,1]]

In [ ]:
# We define the initial conditions using the real-world data to match defined above.
# This example is for a system with equations for CRH, ACTH and Cortisol. The IC for CRH has been set to 0 to start,
#  but should be changed depending on whether it will be optimized.

y0 = [0, data_to_match[1][0], data_to_match[3][0]]

# For matching data with only cortisol concentrations, use the following line and change the ICs as desired:
#y0 = [0, 10, data_to_match[1][0]]

In [ ]:
# We call the run() method of the optimize module, which will run the parameter optimization given the arguments we pass
# Required arguments are the cost function, the model, and the real-world data we want to match
#
# Optional arguments include the initial conditions we want to optimize (if any), the number of iterations to run,
#  the maximum number of optimization steps per iteration of the algorithm, the algorithm to use (defaults to 
#  differential_evolution), and the popsize to use (larger popsize gives more accurate optimization but is more 
#  computationally expensive) 
#
# For instance, to optimize the IC for CRH, enter the optional argument IC_opt = [0]. This will tell the function that
#  the first index of y0 is being optimized, so in this example that is CRH
opt_pars, simData_all = optimize.run(cost_fun, model, data_to_match, y0, bounds, num_iter=5)

[Back to Top](#top)

# Save Output to File <a name="saveoutput"></a>

In [ ]:
# Change the root filename, this will have the array name appended to it
#  to make the filename of the Excel files, textfiles and graphs
filename_root = 'newModel_output/nelson-patientMean-5-iterations-ICopt'

In [ ]:
# Create the pandas DataFrame object for opt_pars
# Unfortunately, you'll need to type out every parameter name if you want to get column names in your Excel file.
#  Be sure to put them in the order they are defined in the parameter list passed to the model.
#
# NOTE: I've been unable to get this code to work for 1 iteration of parameter optimization (only for opt_pars)
# Hopefully in the near future I'll get it worked out, but for some reason I can't get it to be output as a single row
# with multiple columns
df_opt_pars = pd.DataFrame(opt_pars, columns=['Cost',
                                              'parameter1',
                                              'parameter2'])
# Create the pandas DataFrame object for simData_all
# I've typed out each individual heading for the variables and iterations,
# and assigned the correct column of simData_all to them
df_simData_all = pd.DataFrame(simData_all, columns=['Iteration 1 Time',
                                                    'Iteration 1 CRH',
                                                    'Iteration 1 ACTH',
                                                    'Iteration 1 Cortisol',
                                                    'Iteration 2 Time',
                                                    'Iteration 2 CRH',
                                                    'Iteration 2 ACTH',
                                                    'Iteration 2 Cortisol',
                                                    'Iteration 3 Time',
                                                    'Iteration 3 CRH',
                                                    'Iteration 3 ACTH',
                                                    'Iteration 3 Cortisol',
                                                    'Iteration 4 Time',
                                                    'Iteration 4 CRH',
                                                    'Iteration 4 ACTH',
                                                    'Iteration 4 Cortisol',
                                                    'Iteration 5 Time',
                                                    'Iteration 5 CRH',
                                                    'Iteration 5 ACTH',
                                                    'Iteration 5 Cortisol'])

# Create an instance of the ExcelWriter class and open the file using a with statement
with pd.ExcelWriter(filename_root+".xlsx") as writer:
    # Define the header format, so that it's bold, text is wrapped, it has a 
    #  colored background and a border
    header_format = writer.book.add_format({
        'bold': True,
        'text_wrap': True,
        'valign': 'top',
        'fg_color': '#D7E4BC',
        'border': 1
    })
    
    # Write the opt_pars array to a sheet in the file, we skip adding in the headers here and add them with the above
    #  format afterwards. We also change the row index to start at 1, rather than 0.
    df_opt_pars.index = list(range(1,len(opt_pars[:,0])+1))
    df_opt_pars.to_excel(writer, sheet_name="Optimized Parameters", startrow=1, header=False)
    
    # Write the simData_all array to another sheet in the file, we skip adding in the headers here and add them with the above
    #  format afterwards. We also disable the row index numbers, as they are not necessary here.
    df_simData_all.to_excel(writer, sheet_name="Simulated Concentration Data", startrow=1, header=False, index=False)
    
    # Loop through each header in opt_pars DataFrame and write to the sheet with formatting
    for col,val in enumerate(df_opt_pars.columns.values):
        # We write in the sheet "Optimized Parameters" in the first row, starting with the second column 
        #  (because of the row indices), using the headers from the DataFrame and the header format we defined above
        writer.sheets["Optimized Parameters"].write(0, col+1, val, header_format)
    
    # Loop through each header in simData_all DataFrame and write to the sheet with formatting
    for col,val in enumerate(df_simData_all.columns.values):
        # We write in the sheet "Simulated Concentration Data" in the first row, starting with the first column 
        #  (because we turned off the row indices), using the headers from the DataFrame and 
        #  the header format we defined above
        writer.sheets["Simulated Concentration Data"].write(0, col, val, header_format)
    

[Back to Top](#top)

# Compute Means and Std Devations of Parameters and Output as Table <a name="paramtable"></a>

In [ ]:
# Create variables for mean and standard deviation for each parameter
parameter1_mean = np.mean(opt_pars[:,1])
parameter1_std = np.std(opt_pars[:,1])
parameter2_mean = np.mean(opt_pars[:,2])
parameter2_std = np.std(opt_pars[:,2])

In [ ]:
# Output those means and std deviations as a table
print(tabulate([["parameter1", "%f +- %f" % (parameter1_mean, parameter1_std)], ["parameter2", "%f +- %f" % (parameter2_mean, parameter2_std)]], headers = ["Parameter", "Mean +- Standard Deviation"]))


[Back to Top](#top)

# Plots <a name="plots"></a>

In [ ]:
# Create an instance of the Visualizer class from the VeVaPy module called visualize (and pass the dictionary containing
#  the variables in the global namespace so the object has access to them for graphing). This will prompt the user for 
#  several inputs regarding the graphs to generate
grapher = Visualizer(globals())

In [ ]:
# Calling the make_graphs method of the object will generate and display the graphs. Use the command help(Visualizer)
#  to display information regarding the optional arguments for this method
grapher.make_graphs()

[Back to Top](#top)

# No Optimization Run <a name="no-opt" />

Change the values of y0, authors_params and time steps in the section **Parameters and Initial Conditions** near the top of the notebook.

In [ ]:
# Run the simulation without optimization
data_no_opt = model(authors_params, y0)

[Back to Top](#top)

# Run Optimizations for Multiple Patients <a name="runMultiple" />

In [ ]:
# Enter the patient number to start and end the optimization loop
start_patient = 1
end_patient = 10

for patient in range(start_patient, end_patient+1):
    print(f"\033[1mCurrent Patient: #{patient}\033[0m")
    # Change the data set to cycle through patients in here. Will only work for Nelson TSST data, including subtype
    #  data sets (as the basal data sets are only mean concentrations, not individual patients)
    data_to_match = [nelson.ACTH[:,0], nelson.ACTH[:,patient+1], nelson.cortisol[:,0], nelson.cortisol[:,patient+1]]
    y0 = [1, data_to_match[1][0], data_to_match[3][0]]
    
    opt_pars_tmp, simData_all_tmp = optimize.run(cost_fun, model, data_to_match, y0, bounds, num_iter=5)
    
    if patient == start_patient:
        simData_all_multiple = simData_all_tmp
        opt_pars_multiple = opt_pars_tmp
    else:
        simData_all_multiple = np.hstack((simData_all_multiple, simData_all_tmp))
        opt_pars_multiple = np.hstack((opt_pars_multiple, opt_pars_tmp))
        

[Back to Top](#top)

# Dependencies <a name="dependencies"></a>

In [ ]:
%load_ext watermark

In [ ]:
%watermark --iversions